<img src='otus.png'>

### Pytorch, вычислительные графы.

Pytorch - фреймворк для работы с вычислительными графами. Чаще всего Pytorch называют фреймворком для "глубокого обучения", но это слегка сужает область его применения. 

Пример вычислительного графа для функции $x^2 + xy + (x + y)^2$:

<img src='graph.png'>



**Как осуществлять операции с тензорами (n-мерные массивы) в Pytorch**

In [2]:
import torch
import numpy as np

a_numpy = np.random.randn(100, 1000)   # numpy array
a = torch.tensor(a_numpy)   # torch tensor / can be uploaded in GPU memory for faster computations

In [3]:
b = a ** 2 # element-wise operation

In [4]:
%timeit c = a_numpy.dot(a_numpy.T) # matrix multiplication with itself -- numpy

669 µs ± 41.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [5]:
%timeit c = a.mm(a.t())   # matrix multiplication with itself -- pytorch

886 µs ± 66.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


**Как рассчитывать градиенты для произвольных (дифференцируемых) функций**

In [23]:
from torch.autograd import Variable
# print the gradient of 2x^2 + 5x
x = Variable(torch.ones(2, 2), requires_grad=True)
z = 2 * (x * x) + 5 * x
# run the backpropagation
z.backward(torch.ones(2, 2))
print(x.grad)

tensor([[13., 13.],
        [13., 13.]])


**Как пользоваться интерфейсом модуля torch.nn для быстрого создания НС.**

In [ ]:
import torch.nn as nn

In [ ]:
model = nn.Sequential(nn.Linear(20, 200),
                      nn.ReLU(),
                      nn.Linear(200, 1),
                      nn.Sigmoid()
                     )

In [ ]:
inp = torch.randn(3, 20)
model(inp)

### Как Pytorch упрощает работу с НС.

Мы видели следующие полезные абстракции:

1. Модуль torch.nn
2. Оптимизаторы torch.optim
3. Функции потерь, например torch.nn.BCELoss()



## Пример с MNIST.

In [7]:
import os
import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import datasets, transforms

In [8]:
BATCH_SIZE = 32
NUM_EPOCHS = 10
FOLDER = 'MNIST_data'

if not os.path.exists(FOLDER):
    os.mkdir(FOLDER)

In [9]:
trans = transforms.Compose([transforms.ToTensor()])

In [10]:
# данные
train_set = datasets.MNIST(root=FOLDER, train=True, transform=trans, download=True)
test_set = datasets.MNIST(root=FOLDER, train=False, transform=trans, download=True)

Processing...
Done!


In [11]:
train_set

Dataset MNIST
    Number of datapoints: 60000
    Split: train
    Root Location: MNIST_data
    Transforms (if any): Compose(
                             ToTensor()
                         )
    Target Transforms (if any): None

In [12]:
# итераторы
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=BATCH_SIZE, shuffle=False)

In [13]:
sample_x, sample_y = next(iter(train_loader))
print(sample_x.size())
print(sample_y.size())

torch.Size([32, 1, 28, 28])
torch.Size([32])


In [14]:
# один скрытый слой, на выходе 10 логитов (по числу классов)
model = torch.nn.Sequential(
    nn.Linear(784, 100),
    nn.ReLU(),
    nn.Linear(100, 10),    
)

In [15]:
opt = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [16]:
# процесс тренировки; тест напишите для упражнения
for epoch in range(NUM_EPOCHS):
    # trainning
    av_loss = 0.
    correct = 0.
    for x, y in tqdm.tqdm(train_loader):
        # рассчитываем функцию потерь
        x = x.view(BATCH_SIZE, -1)
        out = model(x)
        loss = criterion(out, y)
        # оптимизация параметров
        opt.zero_grad()
        loss.backward()
        opt.step()
        # подсчет статистики за эпоху
        pred = out.max(1, keepdim=True)[1]
        correct += pred.eq(y.view_as(pred)).sum().item()
        av_loss += loss.item()
    print('Epoch: {}; Accuracy train: {:.3f}%'.format(epoch, correct / len(train_loader.dataset) * 100))

100%|█████████████████████████████████████████████████████████████████████████████| 1875/1875 [00:15<00:00, 119.23it/s]


Epoch: 0; Accuracy train: 89.852%


100%|█████████████████████████████████████████████████████████████████████████████| 1875/1875 [00:15<00:00, 122.00it/s]


Epoch: 1; Accuracy train: 95.118%


100%|█████████████████████████████████████████████████████████████████████████████| 1875/1875 [00:19<00:00, 122.34it/s]


...

Epoch: 7; Accuracy train: 98.505%


100%|█████████████████████████████████████████████████████████████████████████████| 1875/1875 [00:15<00:00, 120.66it/s]


Epoch: 8; Accuracy train: 98.700%


100%|█████████████████████████████████████████████████████████████████████████████| 1875/1875 [00:15<00:00, 121.93it/s]


Epoch: 9; Accuracy train: 98.888%


**Другие оптимизаторы:** https://pytorch.org/docs/stable/optim.html

## 5. Пример с MNIST. Сверточная сеть.

Идея: обработка участка изображения должна проходить независимо от конкретного расположения участка.

<img src='conv.png'>

Визуализация применения фильтров на модельном примере: http://cs231n.github.io/convolutional-networks/

Фильтры: http://cs231n.github.io/understanding-cnn/

Популярные архитектуры сверточных сетей: https://github.com/pytorch/vision/tree/master/torchvision/models

## 6. Прочие примеры DL успехов.

**Автокодировщик**

<img src='AutoEncoder.png'>
<img src='autoencoder_schema.jpg'>

Идея:  
Выделить признаки и закономерности, характеризующие данные. Можно сделать это в пространстве меньшей размерности.  
Сделаем так, чтобы сеть обучалась на некоторых данных выдавать те же самые данные. Но с ограничением, слой автокодировщика должен быть меньше (или больше), чем размерность исходных данных.

Применение автокодировщиков:
* понижение размерности (в нейронной сети нет исходных ограничений в модели, решение более универсальное)
* подавление шума (на входе на изображение добавить шум, на выходе ждать исходное изображение)
* генерация данных (variational autoencoder, нужно в слой кодировщика добавить ограничение - задать распределение)
* другие

https://vdumoulin.github.io/morphing_faces/online_demo.html

## Литература

1. Николенко, Кадурин, Архангельская. Глубокое обучение. Погружение в мир нейронных сетей.
2. Aurélien Géron Hands-on Machine Learning with Scikit-Learn and TensorFlow
3. Гудфеллоу, Бенджио, Курвилль. Глубокое обучение
4. Стэнфордский курс cs231n: http://cs231n.stanford.edu/